### Analytics Goal:
Use keywords extracted from social media account activity and recommended account acceptence/rejection history of a user to recommend him/her/them other interesting social media accounts to follow on the platform.   
### Machine Learning Algorithm:
Used keywords to train Word2Vec model and built profile vectors for user account and candidate account. These profile vectors are then used as features to train Logistic Regression model and predict if a user will follow the candidate account or reject the recommendation.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from functools import reduce
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Word2Vec

In [0]:
# Initiate Spark Session
spark = SparkSession.builder \
            .appName("project") \
            .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
            .config("spark.network.timeout", "7200")\
            .getOrCreate()

In [0]:
# mongodb database details and creds
database = 'tencent'
collection = 'recologdetail'
user_name = 'cnayak'
password = 'NNcLhP2SdvbiK9X'
address = 'cluster0.mz09y.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [0]:
# AWS s3 details
aws_access_key = '***'
aws_secret_key = '***'
spark._jsc.hadoopConfiguration().set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.1')
#spark._jsc.hadoopConfiguration().set('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') 
spark._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_key)

In [0]:
spark._jsc.hadoopConfiguration().set('spark.network.timeout','7200s')
spark._jsc.hadoopConfiguration().set('spark.executor.heartbeatInterval','1200s')

#### Load data from mongodb


In [0]:
df = spark.read.format('mongo').option("uri",connection_string).load()

In [0]:
# Select keywords list for candidate accounts
item_kws  = df.select('Keywords_list').distinct()

In [0]:
 # In case there are non-integer type to be converted.
def IntegerSafe(value):
    try:
        return int(value)
    except:
        return None

In [0]:
# define schema for user keywords file
user_kw_schema = StructType([StructField("UserId", IntegerType(), False),\
    StructField("Keywords_weights", StringType(), True)])

In [0]:
# Load file containing keywords extracted from user social media activity like tweets, retweets, comments
user_kw_rdd = sc.textFile('s3://msds697-group18-2022/track1/user_key_word.txt')\
                .map(lambda x: x.split('\t'))\
                .map(lambda x : (IntegerSafe(x[0]), x[1]))

In [0]:
# convert RDD to dataframe
user_kw_df = spark.createDataFrame(user_kw_rdd, user_kw_schema)
user_kw_df.printSchema()

root
-- UserId: integer (nullable = false)
-- Keywords_weights: string (nullable = true)

In [0]:
user_kw_df.show()

+-------+--------------------+
 UserId| Keywords_weights|
+-------+--------------------+
1000000|183:0.6673;2:0.35...|
1000001| 92:1.0|
1000002| 112:1.0|
1000003|154435:0.746;30:0...|
1000004| 118:1.0|
1000005|157:0.484;25:0.43...|
1000006|277:0.7815;1980:0...|
1000007|4069:0.6678;2557:...|
1000008|16:0.7164;154:0.3...|
1000009| 2:1.0|
 100001|1921:0.3853;25231...|
1000010|25:0.4024;26:0.35...|
1000011| 1:1.0|
1000012|84:0.6406;2:0.381...|
1000013|573:0.8168;1323:0...|
1000014|2:0.673;17:0.3486...|
1000015|79561:0.3296;55:0...|
1000016| 12:1.0|
1000017| 17704:1.0|
1000018| 72:1.0|
+-------+--------------------+
only showing top 20 rows

In [0]:
# get list of keywords for user account
def get_keywords(keywords):
    
    '''
    convert string Keywords weights in format "183:0.6673;2:0.35" 
    to  Arrays of keywords --> [183,2]
    '''
    kw_ws = [ x for x in keywords.split(';') ]
    kws = [ IntegerSafe(x.split(":")[0]) for x in kw_ws]
    return kws

In [0]:
# Register function for unwrapping of keywords into Array as UDF
get_kws = udf(get_keywords, ArrayType(IntegerType()))

In [0]:
user_kw_df = user_kw_df.withColumn('Keywords_list', get_kws('Keywords_weights'))

In [0]:
user_kw_df.show()

+-------+--------------------+--------------------+
 UserId| Keywords_weights| Keywords_list|
+-------+--------------------+--------------------+
1000000|183:0.6673;2:0.35...|[183, 2, 359, 363...|
1000001| 92:1.0| [92]|
1000002| 112:1.0| [112]|
1000003|154435:0.746;30:0...|[154435, 30, 220,...|
1000004| 118:1.0| [118]|
1000005|157:0.484;25:0.43...|[157, 25, 198, 18...|
1000006|277:0.7815;1980:0...|[277, 1980, 146, ...|
1000007|4069:0.6678;2557:...| [4069, 2557, 137]|
1000008|16:0.7164;154:0.3...|[16, 154, 164, 24...|
1000009| 2:1.0| [2]|
 100001|1921:0.3853;25231...|[1921, 25231, 718...|
1000010|25:0.4024;26:0.35...|[25, 26, 31, 71, ...|
1000011| 1:1.0| [1]|
1000012|84:0.6406;2:0.381...|[84, 2, 10, 7307,...|
1000013|573:0.8168;1323:0...| [573, 1323, 14]|
1000014|2:0.673;17:0.3486...|[2, 17, 58, 4, 6,...|
1000015|79561:0.3296;55:0...|[79561, 55, 61, 2...|
1000016| 12:1.0| [12]|
1000017| 17704:1.0| [17704]|
1000018| 72:1.0| [72]|
+-------+--------------------+--------------------+
only showing top 20 rows

In [0]:
# combine user account keywords and item account keywords to train the Word2vec 
keywords_all = item_kws.unionByName(user_kw_df.select('Keywords_list'))

In [0]:
keywords_all.count()

Out[22]: 2325602

In [0]:
# cast encoded keywords into string. requirement of Word2Vec model
def cast_keywords(keywords_list):
    return [str(x) for x in keywords_list]

In [0]:
# Register function for casting keywords to string
cast_kws = udf(cast_keywords, ArrayType(StringType()))

In [0]:
keywords_all = keywords_all.withColumn('Keywords', cast_kws('Keywords_list'))

In [0]:
keywords_all.show()

+--------------------+--------------------+
 Keywords_list| Keywords|
+--------------------+--------------------+
[134231, 673391, ...|[134231, 673391, ...|
[254337, 334106, ...|[254337, 334106, ...|
[39017, 3971, 515...|[39017, 3971, 515...|
[355993, 297361, ...|[355993, 297361, ...|
[6359, 19954, 229...|[6359, 19954, 229...|
[6359, 61926, 316...|[6359, 61926, 316...|
[6359, 295444, 24...|[6359, 295444, 24...|
[12860, 15298, 99...|[12860, 15298, 99...|
[594764, 514049, ...|[594764, 514049, ...|
[100947, 360562, ...|[100947, 360562, ...|
[6359, 10697, 618...|[6359, 10697, 618...|
[99, 448344, 317,...|[99, 448344, 317,...|
[254337, 195099, ...|[254337, 195099, ...|
[48583, 602480, 3...|[48583, 602480, 3...|
[619870, 242039, ...|[619870, 242039, ...|
[555948, 638452, ...|[555948, 638452, ...|
[3697, 267001, 47...|[3697, 267001, 47...|
[675933, 524696, ...|[675933, 524696, ...|
[6359, 80509, 320...|[6359, 80509, 320...|
[6359, 12152, 618...|[6359, 12152, 618...|
+--------------------+--------------------+
only showing top 20 rows

#### initialize Word2Vec model. embedding size is set to 5


In [0]:
word2Vec = Word2Vec(vectorSize=5, seed=42, inputCol="Keywords", outputCol="model")

In [0]:
# train the Word2Vec model for 10 iterations
word2Vec.setMaxIter(10)
model = word2Vec.fit(keywords_all)

In [0]:
# Embeddings for keywords
model.getVectors().show()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 warnings.warn(
+------+--------------------+
 word| vector|
+------+--------------------+
 45717|[-0.7876932621002...|
 10292|[-0.3576041460037...|
 19125|[-0.4223117232322...|
 67521|[-0.1706112623214...|
 45017|[-0.4092124998569...|
 26985|[-0.3862453699111...|
 5451|[-0.2081219404935...|
 46826|[-0.3541896939277...|
 4018|[-0.3182195425033...|
 58451|[-0.3585941195487...|
 31211|[-0.4949035644531...|
 17319|[0.04135286435484...|
 20778|[-0.7178880572319...|
 61947|[0.24548031389713...|
 75120|[-0.6012022495269...|
 25285|[-0.6163133382797...|
 65138|[-0.0281073171645...|
300932|[-0.6521722078323...|
291876|[-0.9528971314430...|
 79342|[-0.6839565038681...|
+------+--------------------+
only showing top 20 rows

In [0]:
kw_vectors = model.getVectors().collect()

In [0]:
# convert keyword vectors dataframe to dictionary for fast lookup 
keyword_vectors_map = {}
for row in kw_vectors:
    keyword_vectors_map[int(row['word'])] = row['vector']

In [0]:
# select users and recommended accounts acceptance and rejection history 
log_df  = df.select('UserId','ItemId','Result')
data = log_df.withColumn("Result", F.when(log_df['Result']== -1,0).otherwise(log_df['Result']))

In [0]:
# get keywords for user account
data = data.join(user_kw_df.select('UserId','Keywords_list'), 'UserId', 'inner')\
           .withColumnRenamed('Keywords_list', 'User_keywords')

In [0]:
item_kw_df  = df.select('ItemId','Keywords_list').distinct()

In [0]:
# get keywords for candidate account
data = data.join(item_kw_df, 'ItemId', 'inner').withColumnRenamed('Keywords_list', 'Item_keywords')

In [0]:
# rename 'Result' column as label. Requirement for LogisticRegression model
data = data.withColumnRenamed('Result', 'label')

In [0]:
# get all the keyword embeddings for an account and sum them to get profile vector
def get_keyword_vector(keywords):
    default = [0.0,0.0,0.0,0.0, 0.0]
    default_vector = Vectors.dense(default)
    vectors = list(map(lambda x: keyword_vectors_map.get(x, default_vector), keywords))
    final_vector = reduce(lambda x,y: x+y, vectors)
    return final_vector  

In [0]:
# Register the udf to get summation of keyword embeddings
get_kws_vector = udf(get_keyword_vector, VectorUDT())

In [0]:
# get profile vector for User account
data_user_vector = data.withColumn('User_vector', get_kws_vector('User_keywords'))

In [0]:
data_user_vector.show()

+-------+------+-----+--------------------+--------------------+--------------------+
 ItemId|UserId|label| User_keywords| Item_keywords| User_vector|
+-------+------+-----+--------------------+--------------------+--------------------+
1760403|100068| 1|[4193, 6447, 334,...|[45846, 454309, 4...|[-3.4732505311258...|
1760388|100102| 0|[277, 240, 3237, ...|[555948, 638452, ...|[-4.1937828622758...|
1760388|100570| 1| [445854]|[555948, 638452, ...|[0.0,0.0,0.0,0.0,...|
1760403|100618| 1|[293, 59, 100, 52...|[45846, 454309, 4...|[-7.2770457554142...|
1760388|100668| 0| [12, 1104]|[555948, 638452, ...|[-0.4918202459812...|
1760403|101142| 0| [1921, 4263, 23]|[45846, 454309, 4...|[-2.1911464333534...|
1760388|101142| 0| [1921, 4263, 23]|[555948, 638452, ...|[-2.1911464333534...|
1760388|101142| 0| [1921, 4263, 23]|[555948, 638452, ...|[-2.1911464333534...|
1760388|101142| 0| [1921, 4263, 23]|[555948, 638452, ...|[-2.1911464333534...|
1760388|101142| 0| [1921, 4263, 23]|[555948, 638452, ...|[-2.1911464333534...|
1760403|101248| 0|[976, 14, 89517, ...|[45846, 454309, 4...|[-0.8946809992194...|
1760388|101764| 1|[107, 37, 312, 75...|[555948, 638452, ...|[-4.2695779036730...|
1760403|102035| 0| [234]|[45846, 454309, 4...|[-0.0796017348766...|
1760403|102286| 0| [923]|[45846, 454309, 4...|[-0.3087396025657...|
1760388|102391| 0|[16, 246, 17, 340...|[555948, 638452, ...|[-5.4565728157758...|
1760388|102391| 0|[16, 246, 17, 340...|[555948, 638452, ...|[-5.4565728157758...|
1760403|102430| 0|[690, 1092, 285, ...|[45846, 454309, 4...|[-2.5464228112250...|
1760388|102430| 0|[690, 1092, 285, ...|[555948, 638452, ...|[-2.5464228112250...|
1760388|102430| 0|[690, 1092, 285, ...|[555948, 638452, ...|[-2.5464228112250...|
1760388|102430| 0|[690, 1092, 285, ...|[555948, 638452, ...|[-2.5464228112250...|
+-------+------+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
# get profile vector for candidate account
data_all_vectors = data_user_vector.withColumn('Item_vector', get_kws_vector('Item_keywords'))

In [0]:
# combine all the features using vector assembler
va = VectorAssembler(outputCol="features", inputCols=['User_vector','Item_vector'])

In [0]:
lpoints = va.transform(data_all_vectors).select("features", "label")

In [0]:
#split into train and validation set
splits = lpoints.randomSplit([0.8,0.2])
train = splits[0].cache()
valid = splits[1].cache()

In [0]:
train.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
(10,[5,6,7,8,9],[...| 0|
+--------------------+-----+
only showing top 20 rows

In [0]:
# Intialize Logistic regression model. Maximum number of iterations is set to 10
lr = LogisticRegression(regParam=0.01, maxIter=10, fitIntercept=True)
lrmodel = lr.fit(train)

In [0]:
# predict labels for validation dataset
validpredicts = lrmodel.transform(valid)
validpredicts.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|label| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
(10,[5,6,7,8,9],[...| 0|[2.46531080685459...|[0.92167391546930...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.46531080685459...|[0.92167391546930...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.46531080685459...|[0.92167391546930...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.46531080685459...|[0.92167391546930...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.46531080685459...|[0.92167391546930...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.46531080685459...|[0.92167391546930...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.46531080685459...|[0.92167391546930...| 0.0|
(10,[5,6,7,8,9],[...| 1|[2.46531080685459...|[0.92167391546930...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.54088237370606...|[0.92695859140224...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
(10,[5,6,7,8,9],[...| 0|[2.47442735011196...|[0.92232952400083...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
# get coefficients for each feature
print(lrmodel.coefficients)

[0.014966269318764654,0.03297678633796883,0.004759460690468231,0.03135764889334921,-0.02875935192719302,-0.008347326126066618,-0.006427451587076554,-0.005873951836198092,-0.003901535828056451,-0.008220131417213448]

In [0]:
bceval = MulticlassClassificationEvaluator()

In [0]:
# get the accuracy for the Logistic Regression model
bceval.setMetricName("accuracy")
print (bceval.getMetricName() +":" + str(bceval.evaluate(validpredicts)))

accuracy:0.9282694234353019

In [0]:
# get F1 score for the Logistic Regression model
bceval.setMetricName("f1") 
bceval.evaluate(validpredicts)

Out[137]: 0.8937383044219801